I have impemented the PRKNN from "Effective k‑nearest neighbor models for data
classification enhancement" as a class: PRKNNClassifier.

PRKNNCLassifier is compatiable with sklearn's API, meaning it can be used in Pipelines, and has fit() and predict() methods.

I have yet to implement predictproba(), meaning at the moment you cannot calculate ROC_AUC.

In [1]:
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

from PRKNNClassifier.test_class import PRKNeighborsClassifier


In [2]:
# Loading in some data

from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
wine = fetch_ucirepo(id=109) 
  
# data (as pandas dataframes) 
X = wine.data.features 
y = wine.data.targets 

Quick EDA

In [3]:
X.head()

,Alcohol,Malicacid,Ash,Alcalinity_of_ash,Magnesium,Total_phenols,Flavanoids,Nonflavanoid_phenols,Proanthocyanins,Color_intensity,Hue,0D280_0D315_of_diluted_wines,Proline
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [4]:
y.head()

,class
0,1
1,1
2,1
3,1
4,1


In [5]:
X.isna().sum()

Alcohol                         0
Malicacid                       0
Ash                             0
Alcalinity_of_ash               0
Magnesium                       0
Total_phenols                   0
Flavanoids                      0
Nonflavanoid_phenols            0
Proanthocyanins                 0
Color_intensity                 0
Hue                             0
0D280_0D315_of_diluted_wines    0
Proline                         0
dtype: int64

Our wine dataset has no missing data, and features continuous featues. I will scale these using a minMaxScaler().

I have not checked for outliers, which you should really do when using a MinMaxScaler as the tranformation can be distorted, or whether some of the features are logrithmic etc., but for a demo naive usage will suffice.

In [6]:
# Splitting data into training and testing sets.

X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    train_size=0.8, 
    random_state=33
)

In [7]:
prknnPipe = Pipeline(
    [
        ('scaler', MinMaxScaler()),
        ('predictor', PRKNeighborsClassifier())
    ]
)

prknnPipe.fit(X_train, np.ravel(y_train))
y_pred = prknnPipe.predict(X_test)

f1_macro = f1_score(y_test, y_pred, average='macro')
accuracy = accuracy_score(y_test, y_pred)

In [8]:
print(f"F1_macro: {f1_macro}")
print(f"Accuracy: {accuracy}")

F1_macro: 0.9717813051146384
Accuracy: 0.9722222222222222


As mentioned in review.ipynb, each of the 3 PRKNN model versions are implemented as a wrapper around 2 KNN models, one for calculating proximal ratios, and one for prediction.

To ensure that a user can clearly define the parameters for both internal KNN models within all 3 version, the PRKNN takes 4 arguments:
- 'version': defaults to 'standard'
- 'base_knn_parameters': a dict that takes the exact same arguments as sklearn's KNNClassifier(). These parameters are then set indentically across both internal models.
- 'pr_knn_params': a dict for parameters specifically for the proximal ratio calculating internal knn model.
- 'prediction_knn_params': a dict for parameters specifically for the prediction internal knn model.


In [19]:
# example usage for base parameters

base_knn_params = {
    "n_neighbors": 5,
    "n_jobs": 2
}

# now both the proximal ratio and prediction internal knn models will have the exact same parameters.
prknn = PRKNeighborsClassifier(base_knn_params=base_knn_params)


In [21]:
# example of setting the proximal ratio calculating and prediction models seperately. 

pr_knn_params = {
    "n_neighbors": 10,
    "n_jobs": 3
}

predict_knn_params = {
    "n_neighbors": 5,
    "n_jobs": 2
}

prknn = PRKNeighborsClassifier(predict_knn_params=predict_knn_params, pr_knn_params=pr_knn_params)

In [ ]:
# predict and pr knn parameters must both be supplied together.

try:
    prknn = PRKNeighborsClassifier(pr_knn_params=pr_knn_params)
except ValueError as e:
    print(e)

try:
    prknn = PRKNeighborsClassifier(predict_knn_params=pr_knn_params)
except ValueError as e:
    print(e)

both pr and predict knn parameters must be set if base parameters not set.
both pr and predict knn parameters must be set if base parameters not set.
